# Gabriel's Analysis

## Introduction

The following document contains analysis of cybersecurity data, both adapted from existing code found from the internet and written myself in Python.

The following pre-requisites are required and this block should be run first so that the suitable packages are loaded in.

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO
from datetime import datetime

## Analysis of FTP dataset

FTP stands for File Transfer Protocol. It is a network protocol found in the application layer used to facilitate the transferral of computer files between a client and a server on a network.

We first need to import the dataset (which is in comma seperated value format) from Google Drive to our workspace.

In [86]:
orig_url='https://drive.google.com/file/d/1WemTdLui2MkjG_9ykm2_KPhL1FFfKmiL/view'

file_id = orig_url.split('/')[-2]
dwn_url='https://drive.google.com/uc?export=download&id=' + file_id
url = requests.get(dwn_url).text
csv_raw = StringIO(url)
ftp_df = pd.read_csv(csv_raw)

We have used the "pandas" package to read the csv into the more easily analysable form of a data frame. Next weperform an initial analysis on the dataframe ftp_df.

In [79]:
#DataFrame with columns
columns = pd.DataFrame(list(ftp_df.columns.values[1:]))

#DataFrame with data types
data_types = pd.DataFrame(ftp_df.dtypes, columns=['Data Type'])

#DataFrame with Count
data_count = pd.DataFrame(ftp_df.count(), columns=['Count'])

#DataFrame with unique values
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(ftp_df.columns.values):
    unique_value_counts.loc[v] = [ftp_df[v].nunique()]

missing_data_counts = pd.DataFrame(ftp_df.isnull().sum(), columns=['Missing Values'])
data_quality_report = data_types.join(data_count).join(unique_value_counts).join(missing_data_counts)
print('Data Quality Report')
data_quality_report

Data Quality Report


,Data Type,Count,Unique Values,Missing Values
ts,float64,5796,2390,0
uid,object,5796,137,0
id.orig_h,object,5796,15,0
id.orig_p,int64,5796,95,0
id.resp_h,object,5796,21,0
id.resp_p,int64,5796,1,0
user,object,5796,4,0
password,object,5745,12,51
command,object,5796,6,0
arg,object,2966,1545,2830


To get an impression of what the data looks like, and in doing so check that we have correctly imported the data, we use the head command in order to get a snapshot of the first few terms of our dataframe and see what kind of data is stored in it.

In [70]:
ftp_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
0,1.331904e+09,CNFo204HUpVHDn1qt2,192.168.203.45,34433,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,219,204).",T,192.168.203.45,192.168.21.101,56268.0,NaN
1,1.331904e+09,CyHkLo2YfhjddpbSVl,192.168.203.45,56158,192.168.21.103,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,103,192,28)",T,192.168.203.45,192.168.21.103,49180.0,NaN
2,1.331904e+09,CotBpLi55vt2fNqm7,192.168.202.96,40138,192.168.28.101,21,<unknown>,NaN,PORT,\x1d\x93!\xf8t\x1ck\xd64\x05\xbb\xbeyu$\x152\x...,NaN,NaN,220.0,ProFTPD 1.3.4rc2 Server (Debian) [::ffff:172.1...,NaN,NaN,NaN,NaN,NaN
3,1.331904e+09,CptK3340W66OKHK3Rd,192.168.202.96,43740,192.168.28.103,21,<unknown>,NaN,PORT,\xbe\xb9wN\x11\xd1\xe1yH\xb8fI\x86\xfdvG\x80\x...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN
4,1.331904e+09,C3NlQu4G9w4W3TGSj7,192.168.204.45,50584,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,163,245).",T,192.168.204.45,192.168.21.101,41973.0,NaN


From looking at the data itself and from experience of looking at similar data from examples we recognize that as this data is logging various connections, the first column should be a timestamp of when each connection is initiated. This makes sense as the first column is labelled ts, obviously short for timestamp, however the data in the table doesn't look much like a timestamp at all. This is due to the way the data has been formatted, however we can use the python package datetime to fix this.

In [91]:
ftp_df["ts"].values[1]
print("This is the format of the ts data before transformation:\n", ftp_df["ts"], "\n\nWith the datatype shown at the bottom.")

This is the format of the ts data before transformation:
 0       1.331904e+09
1       1.331904e+09
2       1.331904e+09
3       1.331904e+09
4       1.331904e+09
            ...     
5791    1.332016e+09
5792    1.332016e+09
5793    1.332016e+09
5794    1.332016e+09
5795    1.332016e+09
Name: ts, Length: 5796, dtype: float64 

With the datatype shown at the bottom.


In [92]:
from datetime import datetime
ftp_df['ts'] = [datetime.fromtimestamp(float(date)) for date in ftp_df['ts'].values]
print("This is the format of the ts data after transformation:\n", ftp_df["ts"], "\n\nWith the datatype shown at the bottom.")

This is the format of the ts data after transformation:
 0      2012-03-16 13:12:38.160
1      2012-03-16 13:12:40.090
2      2012-03-16 13:12:38.950
3      2012-03-16 13:12:41.260
4      2012-03-16 13:25:01.560
                 ...          
5791   2012-03-17 20:21:30.290
5792   2012-03-17 20:21:34.700
5793   2012-03-17 20:21:49.730
5794   2012-03-17 20:21:55.460
5795   2012-03-17 20:22:19.970
Name: ts, Length: 5796, dtype: datetime64[ns] 

With the datatype shown at the bottom.


We now see that by parsing the timestamp column through the above function we now have something readable as a date rather than a long string of numbers, this makes it far nicer for us to read the dataframe. We notice that the data type has changed to a "datetime" format, opposed to the previous "float".

However the readability could still be improved, to do this we will rename the columns to something more user friendly. I have done some research into what each part of the dataframe means and have therefore selected suitable column names accordingly, using the rename() function of the pandas package.

In [93]:
ftp_df.rename(columns = {"ts":"Timestamp","uid":"User ID","id.orig_h":"Originating IP","id.resp_h":"Recieving IP","id.orig_p":"Originating Port","id.resp_p":"Recieving Port"}, inplace = True)
ftp_df.head()

,Timestamp,User ID,Originating IP,Originating Port,Recieving IP,Recieving Port,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
0,2012-03-16 13:12:38.160,CNFo204HUpVHDn1qt2,192.168.203.45,34433,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,219,204).",T,192.168.203.45,192.168.21.101,56268.0,NaN
1,2012-03-16 13:12:40.090,CyHkLo2YfhjddpbSVl,192.168.203.45,56158,192.168.21.103,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,103,192,28)",T,192.168.203.45,192.168.21.103,49180.0,NaN
2,2012-03-16 13:12:38.950,CotBpLi55vt2fNqm7,192.168.202.96,40138,192.168.28.101,21,<unknown>,NaN,PORT,\x1d\x93!\xf8t\x1ck\xd64\x05\xbb\xbeyu$\x152\x...,NaN,NaN,220.0,ProFTPD 1.3.4rc2 Server (Debian) [::ffff:172.1...,NaN,NaN,NaN,NaN,NaN
3,2012-03-16 13:12:41.260,CptK3340W66OKHK3Rd,192.168.202.96,43740,192.168.28.103,21,<unknown>,NaN,PORT,\xbe\xb9wN\x11\xd1\xe1yH\xb8fI\x86\xfdvG\x80\x...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN
4,2012-03-16 13:25:01.560,C3NlQu4G9w4W3TGSj7,192.168.204.45,50584,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,163,245).",T,192.168.204.45,192.168.21.101,41973.0,NaN


## Resources

[Secrepo - source of security data](http://www.secrepo.com/)

[ftp.csv dataset](https://drive.google.com/drive/folders/1qBmJhVqPprD-esGKgtm6VLn_YOpjawJ)

[Stack Overflow resource that helped with downloading files from Google Drive](https://stackoverflow.com/questions/56611698/pandas-how-to-read-csv-file-from-google-drive-public)